# Sign clustering

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2002)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = [16, 9]

## Drawing icons

In [79]:
from unicode_transform import get_icons

In [80]:
get_icons(start=40, end=10000)

## Creating the dataset

In [4]:
from image_distance import phash
from cv2 import cv2
import os

In [5]:
prj_path = os.path.dirname(os.path.abspath(os.curdir))

In [77]:
generated_path = os.path.join(prj_path, 'icons', 'generated')
clustered_path = os.path.join(prj_path, 'icons', 'clustered')
img_count = len(os.listdir(generated_path))

### 1D dataset

In [36]:
X = np.zeros((img_count, 1), dtype=np.uint64)
names = []

In [37]:
for i, filepath in enumerate(os.listdir(generated_path)):
    img = cv2.imread(os.path.join(generated_path, filepath))
    names.append(filepath)
    X[i, 0] = phash(img)

### HashSizeD dataset

In [53]:
hash_size = 8

In [54]:
X = np.zeros((img_count, hash_size * hash_size), dtype=np.uint8)
names = []

In [55]:
for i, filepath in enumerate(os.listdir(generated_path)):
    img = cv2.imread(os.path.join(generated_path, filepath))
    names.append(filepath)
    hashed = phash(img)
    X[i, :] = np.array(
        [(hashed >> i) & 1 for i in range(hash_size * hash_size - 1, -1, -1)], 
        np.uint8)

## Clustering

In [38]:
import gmpy2
from scipy.stats import mode
import time

In [39]:
def metric(x, y):
    return gmpy2.popcount(np.bitwise_xor(gmpy2.mpz(x[0]), gmpy2.mpz(y[0])))

### DBSCAN

In [18]:
from sklearn.cluster import DBSCAN

In [19]:
dbscan = DBSCAN(n_jobs=-1, metric=metric, eps=10, min_samples=2)

In [20]:
%%time
y = dbscan.fit_predict(X)

Wall time: 557 ms


In [21]:
y = dbscan.labels_

In [22]:
mode(y)

ModeResult(mode=array([0], dtype=int64), count=array([4428]))

In [23]:
np.sum(y == mode(y).mode)

4428

In [24]:
clustered = X[y != mode(y).mode[0], :]

### K-Means

In [56]:
from sklearn.cluster import KMeans

In [68]:
kmeans = KMeans(n_jobs=-1, n_clusters=455)

In [69]:
%%time
y = kmeans.fit_predict(X)

Wall time: 7.28 s


In [70]:
%%time
X_dist = kmeans.fit_transform(X)

Wall time: 7.82 s


In [71]:
X_dist.shape

(4428, 455)

In [72]:
representative_sign_idx = np.argmin(X_dist, axis=0)
X_representative = X[representative_sign_idx]

In [73]:
X_representative.shape

(455, 64)

In [78]:
clusters_count = np.max(y) + 1
plt.rcParams["figure.figsize"] = [16, clusters_count // 5 * 3]
clusters = defaultdict(lambda: 0)
fig, axes = plt.subplots(clusters_count // 5, 5)
i = 0
for idx in representative_sign_idx:
    read = plt.imread(os.path.join(generated_path, names[idx]))
    axes[i // 5, i % 5].axis('off')
    axes[i // 5, i % 5].imshow(read)
    axes[i // 5, i % 5].set_title(f'Cluster {i}', color='white')
    
    plt.imsave(os.path.join(clustered_path, f'img{i}.jpg'), read)
    
    i += 1
plt.rcParams["figure.figsize"] = [16, 9]

# Showing different clusters

In [ ]:
from collections import defaultdict

In [75]:
plt.rcParams["figure.figsize"] = [16, clusters_count // 5 * 3]
clusters = defaultdict(lambda: 0)
clusters_count = np.max(y)
fig, axes = plt.subplots(clusters_count // 5, 5)
i = 0
idx = -1
while i < clusters_count // 5 * 5:
    idx += 1
    clusters[y[idx]] = clusters[y[idx]] + 1
    if clusters[y[idx]] > 1:
        continue
    read = plt.imread(os.path.join(generated_path, names[idx]))
    axes[i // 5, i % 5].axis('off')
    axes[i // 5, i % 5].imshow(read)
    axes[i // 5, i % 5].set_title(f'Cluster {i}', color='white')
    i += 1
plt.rcParams["figure.figsize"] = [16, 9]

In [ ]:
np.savetxt('X.csv', X, de)